In [1]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
from preprocessing import *

In [2]:
# 메모 (2021.08.04)
# AT_BSC_CVR_CD : 01=대인1, 02=대인2, 03=대물, 04=자기신체/자동차상해, 05=무보험, 06=자기차량, 07=기계장치, 08=다른자동차운전, 09=SOS, 10=기본담보
# AT_PDGR_CD : 01=개인용, 02=업무용, 03=영업용, 04=이륜차, 05=외화표시, 06=운전자, 07=취급업자, 08=모터바이크, 09=농기계, 10=운전면허교습생, 11=회사단독
# AT_CHN_DVCD : 01=전통, 02=GA, 03=TM, 04=CM

In [3]:
# 환경설정
pd.options.display.float_format = '{:,.0f}'.format
os.makedirs('result', exist_ok=True)

# 전역변수
FILE_PATH = Path('./data/현행추정부채_요구자본_자동차/20210804')
BASE_YYMM = '201912'

In [44]:
# 데이터 불러오기
## 보험료부채 익스포져
자동차_원수_미경과보험료 = pd.read_excel(FILE_PATH / f'자동차_원수_미경과보험료_{BASE_YYMM}.xlsx',
    dtype={'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str, 'INER_CHN_DVCD': str, 'USTK_DVCD': str, 'CMSN_DVCD': str, 'OGL_URND_PRM': float})
자동차_비례출재_미경과보험료 = pd.read_excel(FILE_PATH / f'자동차_비례출재_미경과보험료_{BASE_YYMM}.xlsx',
    dtype={'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str, 'INER_CHN_DVCD': str, 'USTK_DVCD': str, 'CMSN_DVCD': str, 'RN_URND_PRM': float})
자동차_비비례출재_미경과보험료 = pd.read_excel(FILE_PATH / f'자동차_비비례출재_미경과보험료_{BASE_YYMM}.xlsx',
    dtype={'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str, 'SL_CVR_CD': str, 'T02_RN_RINSC_CD': str, 'CMSN_DVCD': str, 'RN_URND_PRM': float})
자동차_예정보험료 = pd.read_excel(FILE_PATH / f'자동차_예정보험료_{BASE_YYMM}.xlsx',
    dtype={'PDC_CD': str, 'PDGR_CD': str, 'BSC_CVR_CD': str, 'USTK_DVCD': str, 'INER_CHN_DVCD': str, 'OGL_EPCT_PRM': float, 'RN_EPCT_PRM': float})
자동차_원수_직전3년보험금손조비 = pd.read_excel(FILE_PATH / f'자동차_원수_직전3년보험금손조비_{BASE_YYMM}.xlsx',
    dtype={'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str, 'INER_CHN_DVCD': str, 'OGL_PYN_BNF': float, 'OGL_ACCD_IVMT': float})
자동차_비례출재_직전3년보험금손조비 = pd.read_excel(FILE_PATH / f'자동차_비례출재_직전3년보험금손조비_{BASE_YYMM}.xlsx',
    dtype={'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str, 'INER_CHN_DVCD': str, 'RN_PYN_BNF': float, 'RN_ACCD_IVMT': float})
자동차_비비례출재_직전3년보험금손조비 = pd.read_excel(FILE_PATH / f'자동차_비비례출재_직전3년보험금손조비_{BASE_YYMM}.xlsx',
    dtype={'PDC_CD': str, 'T02_RN_RINSC_CD': str, 'RN_PYN_BNF': float, 'RN_ACCD_IVMT': float})
자동차_원수_직전1년경과보험료 = pd.read_excel(FILE_PATH / f'자동차_원수_직전1년경과보험료_{BASE_YYMM}.xlsx',
      dtype={'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str, 'INER_CHN_DVCD': str, 'USTK_DVCD': str, 'CMSN_DVCD': str, 'OGL_ELP_PRM': float})
자동차_비례출재_직전1년경과보험료 = pd.read_excel(FILE_PATH / f'자동차_비례출재_직전1년경과보험료_{BASE_YYMM}.xlsx',
    dtype={'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str, 'INER_CHN_DVCD': str, 'USTK_DVCD': str, 'CMSN_DVCD': str, 'RN_ELP_PRM': float})
자동차_비비례출재_직전1년경과보험료 = pd.read_excel(FILE_PATH / f'자동차_비비례출재_직전1년경과보험료_{BASE_YYMM}.xlsx',
    dtype={'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str, 'SL_CVR_CD': str, 'T02_RN_RINSC_CD': str, 'CMSN_DVCD': str, 'RN_ELP_PRM': float})

## 준비금부채 익스포져
자동차_원수_개별추산액 = pd.read_excel(FILE_PATH / f'자동차_원수_개별추산액_{BASE_YYMM}.xlsx',
    dtype={'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str, 'INER_CHN_DVCD': str, 'USTK_DVCD': str, 'CMSN_DVCD': str, 'OGL_OST_AMT': float})
자동차_비례출재_개별추산액 = pd.read_excel(FILE_PATH / f'자동차_비례출재_개별추산액_{BASE_YYMM}.xlsx',
    dtype={'BSC_CVR_CD': str, 'PDGR_CD': str, 'PDC_CD': str, 'INER_CHN_DVCD': str, 'CMSN_DVCD': str, 'RN_OST_AMT': float})
자동차_비비례출재_개별추산액 = pd.read_excel(FILE_PATH / f'자동차_비비례출재_개별추산액_{BASE_YYMM}.xlsx',
    dtype={'PDC_CD': str, 'T02_RN_RINSC_CD': str, 'CMSN_DVCD': str, 'RN_OST_AMT': float})

## 기초정보, 가정 등
# 자동차_코드맵핑 = pd.read_excel(FILE_PATH / '자동차_코드맵핑.xlsx', 
#     dtype={'PDGR_CD': str, 'BSC_CVR_CD': str, 'INER_CHN_DVCD': str, 'USTK_DVCD': str, 'AT_PDGR_CD': str, 'AT_BSC_CVR_CD': str, 'AT_CHN_DVCD': str})
# 자동차_코드맵핑.to_csv(FILE_PATH / '자동차_코드맵핑.csv', index=False)
자동차_코드맵핑 = pd.read_csv(FILE_PATH / '자동차_코드맵핑.csv',
    dtype={'PDC_CD': str, 'PDGR_CD': str, 'BSC_CVR_CD': str, 'INER_CHN_DVCD': str, 'USTK_DVCD': str, 'AT_PDGR_CD': str, 'AT_BSC_CVR_CD': str, 'AT_CHN_DVCD': str})
자동차_상품군정보 = pd.read_excel(FILE_PATH / '자동차_상품군정보.xlsx',
    dtype={'PDGR_CD': str, 'BSC_CVR_CD': str, 'BOZ_CD': str})
자동차_보험금진전추이 = pd.read_excel(FILE_PATH / f'자동차_보험금진전추이_{201812}.xlsx',
    dtype={'BOZ_CD': str, 'AY_YM': str, 'BASE_1': float, 'BASE_2': float, 'BASE_3': float, 'BASE_4': float, 'BASE_5': float, 'BASE_6': float, 'BASE_7': float}) \
    .assign(AY = lambda x: x['AY_YM'].str.slice(0,4)).drop('AY_YM', axis=1)
자동차_최종손해율 = pd.read_excel(FILE_PATH / f'자동차_최종손해율_{BASE_YYMM}.xlsx',
    dtype={'RRNR_DVCD': str, 'AT_CHN_DVCD': str, 'AT_PDGR_CD': str, 'FNAL_LSRT': float}) \
    .drop('RRNR_DAT_DVCD', axis=1)
자동차_사업비율 = pd.read_excel(FILE_PATH / f'자동차_사업비율_{BASE_YYMM}.xlsx',
    dtype={'RRNR_DVCD': str, 'BOZ_CD': str, 'EXP_RT': float})
자동차_IBNR = pd.read_excel(FILE_PATH / f'자동차_IBNR_{BASE_YYMM}.xlsx',
    dtype={'RRNR_DVCD': str, 'BSC_CVR_CD': str, 'KRW_OS_AMT_E': float})

## 공통
재보험자_국내신용등급 = pd.read_excel(FILE_PATH / f'재보험자_국내신용등급_{202012}.xlsx',
    dtype={'RINSC_CD': str, 'RINSC_NM': str, 'CRD_GRD': str})
재보험자_KICS신용등급 = pd.read_excel(FILE_PATH / f'재보험자_KICS신용등급_{202012}.xlsx',
    dtype={'RINSC_CD': str, 'RINSC_NM': str, 'KICS_CRDT_GRCD': str}) \
    .rename(columns= {'KICS_CRDT_GRCD': 'KICS_CRD_GRD'})
재보험자_수정 = pd.read_excel(FILE_PATH / f'재보험자_수정_{202012}.xlsx',
    dtype={'T02_RN_RINSC_CD': str, 'RINSC_CD': str})
선도부도율 = pd.read_excel(FILE_PATH / f'선도부도율_{202012}.xlsx',
    dtype={'BASE_YYMM': str, 'GRADE': str, 'YEAR': float, 'FWD_PD': float})
할인율 = pd.read_excel(FILE_PATH / f'할인율_{202012}.xlsx',
    dtype={'KICS_SCEN_NO': int, 'MAT_TERM': int, 'SPOT_RATE': float})

In [48]:
# 데이터 전처리
## 미경과보험료
자동차_원수_미경과보험료_가공 = 자동차_원수_미경과보험료.copy()
자동차_원수_미경과보험료_가공 = 자동차_원수_미경과보험료_가공.merge(자동차_코드맵핑, on=['PDC_CD', 'PDGR_CD', 'BSC_CVR_CD', 'INER_CHN_DVCD', 'USTK_DVCD'], how='left')
자동차_원수_미경과보험료_가공['RRNR_DVCD'] = '01'
자동차_원수_미경과보험료_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_원수_미경과보험료_가공, 자동차_상품군정보)
자동차_원수_미경과보험료_가공['P_NP_DVCD'] = '#'
자동차_원수_미경과보험료_가공['RINSC_CD'] = '#'
자동차_원수_미경과보험료_가공['CRD_GRD'] = '#'
자동차_원수_미경과보험료_가공['KICS_CRD_GRD'] = '#'
자동차_원수_미경과보험료_가공.rename(columns={'OGL_URND_PRM': 'URND_PRM'}, inplace=True)
자동차_원수_미경과보험료_가공 = 자동차_원수_미경과보험료_가공[['RRNR_DVCD', 'BSC_CVR_CD', 'BOZ_CD', 'AT_PDGR_CD', 'AT_CHN_DVCD', 'P_NP_DVCD', 'CMSN_DVCD', 'RINSC_CD', 'CRD_GRD', 'KICS_CRD_GRD', 'URND_PRM']]

자동차_비례출재_미경과보험료_가공 = 자동차_비례출재_미경과보험료.copy()
자동차_비례출재_미경과보험료_가공 = 자동차_비례출재_미경과보험료_가공.merge(자동차_코드맵핑, on=['PDC_CD', 'PDGR_CD', 'BSC_CVR_CD', 'INER_CHN_DVCD', 'USTK_DVCD'], how='left')
자동차_비례출재_미경과보험료_가공['RRNR_DVCD'] = '03'
자동차_비례출재_미경과보험료_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_비례출재_미경과보험료_가공, 자동차_상품군정보)
자동차_비례출재_미경과보험료_가공['P_NP_DVCD'] = 'P'
자동차_비례출재_미경과보험료_가공['RINSC_CD'] = '124012'
자동차_비례출재_미경과보험료_가공['CRD_GRD'] = clsf_crd_grd(자동차_비례출재_미경과보험료_가공, 재보험자_국내신용등급)
자동차_비례출재_미경과보험료_가공['KICS_CRD_GRD'] = clsf_kics_crd_grd(자동차_비례출재_미경과보험료_가공, 재보험자_KICS신용등급)
자동차_비례출재_미경과보험료_가공.rename(columns={'RN_URND_PRM': 'URND_PRM'}, inplace=True)
자동차_비례출재_미경과보험료_가공 = 자동차_비례출재_미경과보험료_가공[['RRNR_DVCD', 'BSC_CVR_CD', 'BOZ_CD', 'AT_PDGR_CD', 'AT_CHN_DVCD', 'P_NP_DVCD', 'CMSN_DVCD', 'RINSC_CD', 'CRD_GRD', 'KICS_CRD_GRD', 'URND_PRM']]

자동차_비비례출재_미경과보험료_가공 = 자동차_비비례출재_미경과보험료.copy()
자동차_비비례출재_미경과보험료_가공['RRNR_DVCD'] = '03'
자동차_비비례출재_미경과보험료_가공['BOZ_CD'] = 'B007' # 수정필요
자동차_비비례출재_미경과보험료_가공['P_NP_DVCD'] = 'N'
자동차_비비례출재_미경과보험료_가공['RINSC_CD'] = clsf_rinsc_cd(자동차_비비례출재_미경과보험료_가공, 재보험자_수정)
자동차_비비례출재_미경과보험료_가공['CRD_GRD'] = clsf_crd_grd(자동차_비비례출재_미경과보험료_가공, 재보험자_국내신용등급)
자동차_비비례출재_미경과보험료_가공['KICS_CRD_GRD'] = clsf_kics_crd_grd(자동차_비비례출재_미경과보험료_가공, 재보험자_KICS신용등급)
자동차_비비례출재_미경과보험료_가공['AT_PDGR_CD'] = '01' # 수정필요 (현재는 미경과보험료 0으로 영향은 없음)
자동차_비비례출재_미경과보험료_가공['AT_CHN_DVCD'] = '01' # 수정필요 (현재는 미경과보험료 0으로 영향은 없음)
자동차_비비례출재_미경과보험료_가공.rename(columns={'RN_URND_PRM': 'URND_PRM'}, inplace=True)
자동차_비비례출재_미경과보험료_가공 = 자동차_비비례출재_미경과보험료_가공[['RRNR_DVCD', 'BSC_CVR_CD', 'BOZ_CD', 'AT_PDGR_CD', 'AT_CHN_DVCD', 'P_NP_DVCD', 'CMSN_DVCD', 'RINSC_CD', 'CRD_GRD', 'KICS_CRD_GRD', 'URND_PRM']]

자동차_미경과보험료_가공 = pd.concat([자동차_원수_미경과보험료_가공, 자동차_비례출재_미경과보험료_가공, 자동차_비비례출재_미경과보험료_가공], axis=0).reset_index(drop=True)

## 예정보험료
자동차_예정보험료_가공 = 자동차_예정보험료.copy()
자동차_예정보험료_가공 = 자동차_예정보험료_가공.merge(자동차_코드맵핑, on=['PDC_CD', 'PDGR_CD', 'BSC_CVR_CD', 'INER_CHN_DVCD', 'USTK_DVCD'], how='left')
자동차_예정보험료_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_예정보험료_가공, 자동차_상품군정보)
자동차_예정보험료_가공['RINSC_CD'] = '124012'
자동차_예정보험료_가공['CRD_GRD'] = clsf_crd_grd(자동차_예정보험료_가공, 재보험자_국내신용등급)
자동차_예정보험료_가공['KICS_CRD_GRD'] = clsf_kics_crd_grd(자동차_예정보험료_가공, 재보험자_KICS신용등급)

자동차_원수_예정보험료_가공 = 자동차_예정보험료_가공.drop('RN_EPCT_PRM', axis=1)
자동차_원수_예정보험료_가공.rename(columns={'OGL_EPCT_PRM': 'EPCT_PRM'}, inplace=True)
자동차_원수_예정보험료_가공['RRNR_DVCD'] = '01'
자동차_원수_예정보험료_가공 = 자동차_원수_예정보험료_가공[['RRNR_DVCD', 'BSC_CVR_CD', 'BOZ_CD', 'AT_PDGR_CD', 'AT_CHN_DVCD', 'RINSC_CD', 'CRD_GRD', 'KICS_CRD_GRD', 'EPCT_PRM']]

자동차_비례출재_예정보험료_가공 = 자동차_예정보험료_가공.drop('OGL_EPCT_PRM', axis=1)
자동차_비례출재_예정보험료_가공.rename(columns={'RN_EPCT_PRM': 'EPCT_PRM'}, inplace=True)
자동차_비례출재_예정보험료_가공['RRNR_DVCD'] = '03'
자동차_비례출재_예정보험료_가공 = 자동차_비례출재_예정보험료_가공[['RRNR_DVCD', 'BSC_CVR_CD', 'BOZ_CD', 'AT_PDGR_CD', 'AT_CHN_DVCD', 'RINSC_CD', 'CRD_GRD', 'KICS_CRD_GRD', 'EPCT_PRM']]

자동차_예정보험료_가공 = pd.concat([자동차_원수_예정보험료_가공, 자동차_비례출재_예정보험료_가공], axis=0).reset_index(drop=True)

## 직전1년경과보험료
자동차_원수_직전1년경과보험료_가공 = 자동차_원수_직전1년경과보험료.copy()
자동차_원수_직전1년경과보험료_가공 = 자동차_원수_직전1년경과보험료_가공.merge(자동차_코드맵핑, on=['PDC_CD', 'PDGR_CD', 'BSC_CVR_CD', 'INER_CHN_DVCD', 'USTK_DVCD'], how='left')
자동차_원수_직전1년경과보험료_가공['RRNR_DVCD'] = '01'
자동차_원수_직전1년경과보험료_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_원수_직전1년경과보험료_가공, 자동차_상품군정보)
자동차_원수_직전1년경과보험료_가공['P_NP_DVCD'] = '#'
자동차_원수_직전1년경과보험료_가공['RINSC_CD'] = '#'
자동차_원수_직전1년경과보험료_가공['CRD_GRD'] = '#'
자동차_원수_직전1년경과보험료_가공['KICS_CRD_GRD'] = '#'
자동차_원수_직전1년경과보험료_가공.rename(columns={'OGL_ELP_PRM': 'ELP_PRM'}, inplace=True)
자동차_원수_직전1년경과보험료_가공 = 자동차_원수_직전1년경과보험료_가공[['RRNR_DVCD', 'BSC_CVR_CD', 'BOZ_CD', 'AT_PDGR_CD', 'AT_CHN_DVCD', 'P_NP_DVCD', 'CMSN_DVCD', 'RINSC_CD', 'CRD_GRD', 'KICS_CRD_GRD', 'ELP_PRM']]

자동차_비례출재_직전1년경과보험료_가공 = 자동차_비례출재_직전1년경과보험료.copy()
자동차_비례출재_직전1년경과보험료_가공 = 자동차_비례출재_직전1년경과보험료_가공.merge(자동차_코드맵핑, on=['PDC_CD', 'PDGR_CD', 'BSC_CVR_CD', 'INER_CHN_DVCD', 'USTK_DVCD'], how='left')
자동차_비례출재_직전1년경과보험료_가공['RRNR_DVCD'] = '03'
자동차_비례출재_직전1년경과보험료_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_비례출재_직전1년경과보험료_가공, 자동차_상품군정보)
자동차_비례출재_직전1년경과보험료_가공['P_NP_DVCD'] = 'P'
자동차_비례출재_직전1년경과보험료_가공['RINSC_CD'] = '124012'
자동차_비례출재_직전1년경과보험료_가공['CRD_GRD'] = clsf_crd_grd(자동차_비례출재_직전1년경과보험료_가공, 재보험자_국내신용등급)
자동차_비례출재_직전1년경과보험료_가공['KICS_CRD_GRD'] = clsf_kics_crd_grd(자동차_비례출재_직전1년경과보험료_가공, 재보험자_KICS신용등급)
자동차_비례출재_직전1년경과보험료_가공.rename(columns={'RN_ELP_PRM': 'ELP_PRM'}, inplace=True)
자동차_비례출재_직전1년경과보험료_가공 = 자동차_비례출재_직전1년경과보험료_가공[['RRNR_DVCD', 'BSC_CVR_CD', 'BOZ_CD', 'AT_PDGR_CD', 'AT_CHN_DVCD', 'P_NP_DVCD', 'CMSN_DVCD', 'RINSC_CD', 'CRD_GRD', 'KICS_CRD_GRD', 'ELP_PRM']]

자동차_비비례출재_직전1년경과보험료_가공 = 자동차_비비례출재_직전1년경과보험료.copy()
자동차_비비례출재_직전1년경과보험료_가공['RRNR_DVCD'] = '03'
자동차_비비례출재_직전1년경과보험료_가공['BOZ_CD'] = 'B007' # 수정필요
자동차_비비례출재_직전1년경과보험료_가공['P_NP_DVCD'] = 'N'
자동차_비비례출재_직전1년경과보험료_가공['RINSC_CD'] = clsf_rinsc_cd(자동차_비비례출재_직전1년경과보험료_가공, 재보험자_수정)
자동차_비비례출재_직전1년경과보험료_가공['CRD_GRD'] = clsf_crd_grd(자동차_비비례출재_직전1년경과보험료_가공, 재보험자_국내신용등급)
자동차_비비례출재_직전1년경과보험료_가공['KICS_CRD_GRD'] = clsf_kics_crd_grd(자동차_비비례출재_직전1년경과보험료_가공, 재보험자_KICS신용등급)
자동차_비비례출재_직전1년경과보험료_가공['AT_PDGR_CD'] = '01' # 수정필요
자동차_비비례출재_직전1년경과보험료_가공['AT_CHN_DVCD'] = '01' # 수정필요
자동차_비비례출재_직전1년경과보험료_가공.rename(columns={'RN_ELP_PRM': 'ELP_PRM'}, inplace=True)
자동차_비비례출재_직전1년경과보험료_가공 = 자동차_비비례출재_직전1년경과보험료_가공[['RRNR_DVCD', 'BSC_CVR_CD', 'BOZ_CD', 'AT_PDGR_CD', 'AT_CHN_DVCD', 'P_NP_DVCD', 'CMSN_DVCD', 'RINSC_CD', 'CRD_GRD', 'KICS_CRD_GRD', 'ELP_PRM']]

자동차_직전1년경과보험료_가공 = pd.concat([자동차_원수_직전1년경과보험료_가공, 자동차_비례출재_직전1년경과보험료_가공, 자동차_비비례출재_직전1년경과보험료_가공], axis=0).reset_index(drop=True)

## 개별추산액
자동차_원수_개별추산액_가공 = 자동차_원수_개별추산액.copy()
자동차_원수_개별추산액_가공['RRNR_DVCD'] = '01'
자동차_원수_개별추산액_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_원수_개별추산액_가공, 자동차_상품군정보)
자동차_원수_개별추산액_가공['P_NP_DVCD'] = '#'
자동차_원수_개별추산액_가공['RINSC_CD'] = '#'
자동차_원수_개별추산액_가공['CRD_GRD'] = '#'
자동차_원수_개별추산액_가공['KICS_CRD_GRD'] = '#'
자동차_원수_개별추산액_가공.rename(columns={'OGL_OST_AMT': 'OST_AMT'}, inplace=True)
자동차_원수_개별추산액_가공 = 자동차_원수_개별추산액_가공[['RRNR_DVCD', 'BOZ_CD', 'RINSC_CD', 'CRD_GRD', 'KICS_CRD_GRD', 'OST_AMT']]

자동차_비례출재_개별추산액_가공 = 자동차_비례출재_개별추산액.copy()
자동차_비례출재_개별추산액_가공['RRNR_DVCD'] = '03'
자동차_비례출재_개별추산액_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_비례출재_개별추산액_가공, 자동차_상품군정보)
자동차_비례출재_개별추산액_가공['P_NP_DVCD'] = 'P'
자동차_비례출재_개별추산액_가공['RINSC_CD'] = '124012'
자동차_비례출재_개별추산액_가공['CRD_GRD'] = clsf_crd_grd(자동차_비례출재_개별추산액_가공, 재보험자_국내신용등급)
자동차_비례출재_개별추산액_가공['KICS_CRD_GRD'] = clsf_kics_crd_grd(자동차_비례출재_개별추산액_가공, 재보험자_KICS신용등급)
자동차_비례출재_개별추산액_가공.rename(columns={'RN_OST_AMT': 'OST_AMT'}, inplace=True)
자동차_비례출재_개별추산액_가공 = 자동차_비례출재_개별추산액_가공[['RRNR_DVCD', 'BOZ_CD', 'P_NP_DVCD', 'CMSN_DVCD', 'RINSC_CD', 'CRD_GRD', 'KICS_CRD_GRD', 'OST_AMT']]

자동차_비비례출재_개별추산액_가공 = 자동차_비비례출재_개별추산액.copy()
자동차_비비례출재_개별추산액_가공['RRNR_DVCD'] = '03'
자동차_비비례출재_개별추산액_가공['BOZ_CD'] = 'B007' # 수정필요
자동차_비비례출재_개별추산액_가공['P_NP_DVCD'] = 'N'
자동차_비비례출재_개별추산액_가공['RINSC_CD'] = clsf_rinsc_cd(자동차_비비례출재_개별추산액_가공, 재보험자_수정)
자동차_비비례출재_개별추산액_가공['CRD_GRD'] = clsf_crd_grd(자동차_비비례출재_개별추산액_가공, 재보험자_국내신용등급)
자동차_비비례출재_개별추산액_가공['KICS_CRD_GRD'] = clsf_kics_crd_grd(자동차_비비례출재_개별추산액_가공, 재보험자_KICS신용등급)
자동차_비비례출재_개별추산액_가공.rename(columns={'RN_OST_AMT': 'OST_AMT'}, inplace=True)
자동차_비비례출재_개별추산액_가공 = 자동차_비비례출재_개별추산액_가공[['RRNR_DVCD', 'BOZ_CD', 'P_NP_DVCD', 'CMSN_DVCD', 'RINSC_CD', 'CRD_GRD', 'KICS_CRD_GRD', 'OST_AMT']]

자동차_개별추산액_가공 = pd.concat([자동차_원수_개별추산액_가공, 자동차_비례출재_개별추산액_가공, 자동차_비비례출재_개별추산액_가공], axis=0).reset_index(drop=True)

## 직전3년보험금손조비
자동차_원수_직전3년보험금손조비_가공 = 자동차_원수_직전3년보험금손조비.copy()
자동차_원수_직전3년보험금손조비_가공['RRNR_DVCD'] = '01'
자동차_원수_직전3년보험금손조비_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_원수_직전3년보험금손조비_가공, 자동차_상품군정보)
자동차_원수_직전3년보험금손조비_가공.rename(columns={'OGL_PYN_BNF': 'PYN_BNF', 'OGL_ACCD_IVMT': 'ACCD_IVMT'}, inplace=True)
자동차_원수_직전3년보험금손조비_가공 = 자동차_원수_직전3년보험금손조비_가공[['RRNR_DVCD', 'BOZ_CD', 'PYN_BNF', 'ACCD_IVMT']]

자동차_비례출재_직전3년보험금손조비_가공 = 자동차_비례출재_직전3년보험금손조비.copy()
자동차_비례출재_직전3년보험금손조비_가공['RN_ACCD_IVMT'] = 자동차_비례출재_직전3년보험금손조비_가공['RN_ACCD_IVMT'].fillna(0)
자동차_비례출재_직전3년보험금손조비_가공['RRNR_DVCD'] = '03'
자동차_비례출재_직전3년보험금손조비_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_비례출재_직전3년보험금손조비_가공, 자동차_상품군정보)
자동차_비례출재_직전3년보험금손조비_가공.rename(columns={'RN_PYN_BNF': 'PYN_BNF', 'RN_ACCD_IVMT': 'ACCD_IVMT'}, inplace=True)
자동차_비례출재_직전3년보험금손조비_가공 = 자동차_비례출재_직전3년보험금손조비_가공[['RRNR_DVCD', 'BOZ_CD', 'PYN_BNF', 'ACCD_IVMT']]

자동차_비비례출재_직전3년보험금손조비_가공 = 자동차_비비례출재_직전3년보험금손조비.copy()
자동차_비비례출재_직전3년보험금손조비_가공['RRNR_DVCD'] = '03'
자동차_비비례출재_직전3년보험금손조비_가공['BOZ_CD'] = 'B007' # 수정필요
자동차_비비례출재_직전3년보험금손조비_가공.rename(columns={'RN_PYN_BNF': 'PYN_BNF', 'RN_ACCD_IVMT': 'ACCD_IVMT'}, inplace=True)
자동차_비비례출재_직전3년보험금손조비_가공 = 자동차_비비례출재_직전3년보험금손조비_가공[['RRNR_DVCD', 'BOZ_CD', 'PYN_BNF', 'ACCD_IVMT']]

자동차_직전3년보험금손조비_가공 = pd.concat([자동차_원수_직전3년보험금손조비_가공, 자동차_비례출재_직전3년보험금손조비_가공, 자동차_비비례출재_직전3년보험금손조비_가공], axis=0).reset_index(drop=True)

In [49]:
pd.options.display.float_format = '{:,.10f}'.format

# 가정산출
## 최종손해율
### 변경없음

## 손해조사비율
### 원수, 출재(비례+비비례)로 그룹핑
자동차_손해조사비율 = 자동차_직전3년보험금손조비_가공 \
    .groupby(['RRNR_DVCD', 'BOZ_CD'])[['PYN_BNF', 'ACCD_IVMT']].sum() \
    .eval('LAE_RATIO = ACCD_IVMT/PYN_BNF').reset_index() \
    [['RRNR_DVCD', 'BOZ_CD', 'LAE_RATIO']]

## 할인요소
자동차_할인요소 = []
for 시나리오 in 할인율['KICS_SCEN_NO'].unique():
    자동차_할인요소_개별시나리오 = get_disc_factor_all_c(자동차_보험금진전추이, 할인율.query('KICS_SCEN_NO == @시나리오'))
    자동차_할인요소_개별시나리오.insert(0, 'KICS_SCEN_NO', 시나리오)
    자동차_할인요소.append(자동차_할인요소_개별시나리오)
자동차_할인요소 = pd.concat(자동차_할인요소, axis=0)

# 재보험자산 손실조정율
자동차_손실조정율 = []
for 시나리오 in 할인율['KICS_SCEN_NO'].unique():
    자동차_손실조정율_개별시나리오 = get_loss_adj_rate_all_c(자동차_보험금진전추이, 할인율.query('KICS_SCEN_NO == @시나리오'), 선도부도율)
    자동차_손실조정율_개별시나리오.insert(0, 'KICS_SCEN_NO', 시나리오)
    자동차_손실조정율.append(자동차_손실조정율_개별시나리오)
자동차_손실조정율 = pd.concat(자동차_손실조정율, axis=0)

## 사업비율
### 변경없음

## IBNR
자동차_IBNR_가공 = 자동차_IBNR.copy()
자동차_IBNR_가공.rename(columns={'KRW_OS_AMT_E': 'IBNR_TOT'}, inplace=True)

## 유효만기
자동차_유효만기 = []
for boz_cd in 자동차_보험금진전추이['BOZ_CD'].unique():
    cf_t_prm, cf_rate_prm = get_cf_c(자동차_보험금진전추이.query('BOZ_CD == @boz_cd'), boz_cd, '보험료')
    cf_t_rsv, cf_rate_rsv = get_cf_c(자동차_보험금진전추이.query('BOZ_CD == @boz_cd'), boz_cd, '보험금')
    자동차_유효만기.append([boz_cd, sum(cf_t_prm*cf_rate_prm), sum(cf_t_rsv*cf_rate_rsv)])
자동차_유효만기 = pd.DataFrame(자동차_유효만기, columns=['BOZ_CD', 'EFF_MAT_PRM', 'EFF_MAT_RSV'])

In [50]:
pd.options.display.float_format = '{:,.0f}'.format

# 데이터 집계
## 보험료부채
KEYS = ['RRNR_DVCD', 'AT_PDGR_CD', 'AT_CHN_DVCD', 'RINSC_CD', 'CRD_GRD', 'KICS_CRD_GRD', 'BOZ_CD']
자동차_미경과보험료_집계 = 자동차_미경과보험료_가공.groupby(KEYS)['URND_PRM'].sum().reset_index()
자동차_예정보험료_집계 = 자동차_예정보험료_가공.groupby(KEYS)['EPCT_PRM'].sum().reset_index()
자동차_보험료부채 = 자동차_미경과보험료_집계 \
    .merge(자동차_예정보험료_집계, on=KEYS, how='outer') \
    .merge(자동차_최종손해율, on=['AT_PDGR_CD', 'AT_CHN_DVCD'], how='left') \
    .merge(자동차_사업비율, on=['RRNR_DVCD', 'BOZ_CD'], how='left') \
    .merge(자동차_손해조사비율, on=['RRNR_DVCD', 'BOZ_CD'], how='left') \
    .merge(자동차_할인요소[['KICS_SCEN_NO', 'BOZ_CD', 'DISC_FAC_PRM']], on=['BOZ_CD'], how='left') \
    .merge(자동차_손실조정율[['KICS_SCEN_NO', 'RRNR_DVCD', 'BOZ_CD', 'CRD_GRD', 'LOSS_ADJ_RATE_PRM']], on=['KICS_SCEN_NO', 'RRNR_DVCD', 'BOZ_CD', 'CRD_GRD'], how='left') \
    .merge(자동차_유효만기[['BOZ_CD', 'EFF_MAT_PRM']], on=['BOZ_CD'], how='left') \
    .assign(URND_PRM = lambda x: x.URND_PRM.fillna(0).astype(float)) \
    .assign(EPCT_PRM = lambda x: x.EPCT_PRM.fillna(0).astype(float)) \
    .assign(EXP_RT = lambda x: x['EXP_RT'].fillna(0).astype(float)) \
    .assign(LOSS_ADJ_RATE_PRM = lambda x: x['LOSS_ADJ_RATE_PRM'].fillna(0).astype(float))

자동차_보험료부채 = 자동차_보험료부채 \
        .eval('FNAL_LOSS = (URND_PRM+EPCT_PRM)*FNAL_LSRT') \
        .eval('LAE = FNAL_LOSS*LAE_RATIO') \
        .eval('MAINT_EXP = (URND_PRM+EPCT_PRM)*EXP_RT') \
        .eval('LIAB_PRM = -EPCT_PRM+FNAL_LOSS+LAE+MAINT_EXP') \
        .eval('PV_EPCT_PRM = EPCT_PRM*DISC_FAC_PRM') \
        .eval('PV_FNAL_LOSS = FNAL_LOSS*DISC_FAC_PRM') \
        .eval('PV_LAE = LAE*DISC_FAC_PRM') \
        .eval('PV_MAINT_EXP = MAINT_EXP*DISC_FAC_PRM') \
        .eval('PV_LIAB_PRM = LIAB_PRM*DISC_FAC_PRM') \
        .eval('LOSS_ADJ_PRM = LIAB_PRM*LOSS_ADJ_RATE_PRM')
자동차_보험료부채.loc[lambda x: x['RRNR_DVCD'].isin(['01', '02']), 'RRNR_DVCD'] = '04'
자동차_보험료부채_집계 = 자동차_보험료부채.groupby(['KICS_SCEN_NO', 'RRNR_DVCD', 'BOZ_CD'])[['PV_EPCT_PRM', 'PV_FNAL_LOSS', 'PV_LAE', 'PV_MAINT_EXP', 'PV_LIAB_PRM', 'LOSS_ADJ_PRM']].sum().reset_index()

## 준비금부채
KEYS = ['RRNR_DVCD', 'RINSC_CD', 'CRD_GRD', 'KICS_CRD_GRD', 'BOZ_CD']
자동차_개별추산액_집계 = 자동차_개별추산액_가공 \
        .groupby(KEYS)['OST_AMT'].sum().reset_index()
자동차_미경과보험료_부분합 = 자동차_미경과보험료_가공 \
    .groupby(['RRNR_DVCD', 'BSC_CVR_CD'])['URND_PRM'].sum().reset_index(name='URND_PRM_SUM')
자동차_미경과보험료_비중 = 자동차_미경과보험료_가공 \
    .merge(자동차_미경과보험료_부분합, on=['RRNR_DVCD', 'BSC_CVR_CD'], how='left') \
    .eval('URND_PRM_WGT = URND_PRM/URND_PRM_SUM')
자동차_IBNR_집계 = 자동차_미경과보험료_비중.merge(자동차_IBNR_가공, on=['RRNR_DVCD', 'BSC_CVR_CD'], how='left') \
    .eval('IBNR = IBNR_TOT*URND_PRM_WGT') \
    .groupby(KEYS, as_index=False)['IBNR'].sum()
자동차_준비금부채 = 자동차_개별추산액_집계 \
    .merge(자동차_IBNR_집계, on=KEYS, how='outer') \
    .merge(자동차_손해조사비율, on=['RRNR_DVCD', 'BOZ_CD'], how='left') \
    .merge(자동차_할인요소[['KICS_SCEN_NO', 'BOZ_CD', 'DISC_FAC_RSV']], on=['BOZ_CD'], how='left') \
    .merge(자동차_손실조정율[['KICS_SCEN_NO', 'RRNR_DVCD', 'BOZ_CD', 'CRD_GRD', 'LOSS_ADJ_RATE_RSV']], on=['KICS_SCEN_NO', 'RRNR_DVCD', 'BOZ_CD', 'CRD_GRD'], how='left') \
    .merge(자동차_유효만기[['BOZ_CD', 'EFF_MAT_RSV']], on=['BOZ_CD'], how='left') \
    .assign(OST_AMT = lambda x: x['OST_AMT'].fillna(0).astype(float)) \
    .assign(IBNR = lambda x: x['IBNR'].fillna(0).astype(float)) \
    .assign(LOSS_ADJ_RATE_RSV = lambda x: x['LOSS_ADJ_RATE_RSV'].fillna(0).astype(float)) \
    .eval('FUT_LAE = (OST_AMT*0.5+IBNR*1.0)*LAE_RATIO') \
    .eval('LIAB_RSV = OST_AMT+IBNR+FUT_LAE') \
    .eval('PV_OST_AMT = OST_AMT*DISC_FAC_RSV') \
    .eval('PV_IBNR = IBNR*DISC_FAC_RSV') \
    .eval('PV_FUT_LAE = FUT_LAE*DISC_FAC_RSV') \
    .eval('PV_LIAB_RSV = LIAB_RSV*DISC_FAC_RSV') \
    .eval('LOSS_ADJ_RSV = LIAB_RSV*LOSS_ADJ_RATE_RSV')
자동차_준비금부채.loc[lambda x: x['RRNR_DVCD'].isin(['01', '02']), 'RRNR_DVCD'] = '04'
자동차_준비금부채_집계 = 자동차_준비금부채.groupby(['KICS_SCEN_NO', 'RRNR_DVCD', 'BOZ_CD'])[['PV_OST_AMT', 'PV_IBNR', 'PV_FUT_LAE', 'PV_LIAB_RSV', 'LOSS_ADJ_RSV']].sum().reset_index()

In [51]:
# 신용위험액
## 보험료부채
자동차_할인전보험료부채유효만기_재보험자별 = 자동차_보험료부채.query('KICS_SCEN_NO == 1').query('RRNR_DVCD == "03"') \
    .groupby(['RINSC_CD', 'KICS_CRD_GRD']).apply(lambda x: sum(x['LIAB_PRM']*x['EFF_MAT_PRM'])).reset_index(name='LIAB_PRM_MULT_EFF_MAT') \
    .query('LIAB_PRM_MULT_EFF_MAT != 0')
자동차_할인전보험료부채_재보험자별 = 자동차_보험료부채.query('KICS_SCEN_NO == 1').query('RRNR_DVCD == "03"') \
    .groupby(['RINSC_CD', 'KICS_CRD_GRD']).apply(lambda x: sum(x['LIAB_PRM'])).reset_index(name='LIAB_PRM') \
    .query('LIAB_PRM != 0')
자동차_보험료부채_재보험자별 = 자동차_보험료부채.query('KICS_SCEN_NO == 1').query('RRNR_DVCD == "03"') \
    .groupby(['RINSC_CD', 'KICS_CRD_GRD']).apply(lambda x: sum(x['PV_LIAB_PRM']-x['LOSS_ADJ_PRM'])).reset_index(name='PV_LIAB_PRM_LOSS_ADJ') \
    .query('PV_LIAB_PRM_LOSS_ADJ != 0')
자동차_유효만기_보험료부채_재보험자별 = 자동차_할인전보험료부채_재보험자별 \
    .merge(자동차_보험료부채_재보험자별, on=['RINSC_CD', 'KICS_CRD_GRD'], how='outer') \
    .merge(자동차_할인전보험료부채유효만기_재보험자별, on=['RINSC_CD', 'KICS_CRD_GRD'], how='outer') \
    .eval('EFF_MAT_PRM = LIAB_PRM_MULT_EFF_MAT/LIAB_PRM') \
    .drop(['LIAB_PRM_MULT_EFF_MAT'], axis=1)

## 준비금부채
자동차_할인전준비금부채유효만기_재보험자별 = 자동차_준비금부채.query('KICS_SCEN_NO == 1').query('RRNR_DVCD == "03"') \
    .groupby(['RINSC_CD', 'KICS_CRD_GRD']).apply(lambda x: sum(x['LIAB_RSV']*x['EFF_MAT_RSV'])).reset_index(name='LIAB_RSV_MULT_EFF_MAT') \
    .query('LIAB_RSV_MULT_EFF_MAT != 0')
자동차_할인전준비금부채_재보험자별 = 자동차_준비금부채.query('KICS_SCEN_NO == 1').query('RRNR_DVCD == "03"') \
    .groupby(['RINSC_CD', 'KICS_CRD_GRD']).apply(lambda x: sum(x['LIAB_RSV'])).reset_index(name='LIAB_RSV') \
    .query('LIAB_RSV != 0')
자동차_준비금부채_재보험자별 = 자동차_준비금부채.query('KICS_SCEN_NO == 1').query('RRNR_DVCD == "03"') \
    .groupby(['RINSC_CD', 'KICS_CRD_GRD']).apply(lambda x: sum(x['PV_LIAB_RSV']-x['LOSS_ADJ_RSV'])).reset_index(name='PV_LIAB_RSV_LOSS_ADJ') \
    .query('PV_LIAB_RSV_LOSS_ADJ != 0')
자동차_유효만기_준비금부채_재보험자별 = 자동차_할인전준비금부채_재보험자별 \
    .merge(자동차_준비금부채_재보험자별, on=['RINSC_CD', 'KICS_CRD_GRD'], how='outer') \
    .merge(자동차_할인전준비금부채유효만기_재보험자별, on=['RINSC_CD', 'KICS_CRD_GRD'], how='outer') \
    .eval('EFF_MAT_RSV = LIAB_RSV_MULT_EFF_MAT/LIAB_RSV') \
    .drop(['LIAB_RSV_MULT_EFF_MAT'], axis=1)

## 집계
자동차_유효만기_재보험자별 = 자동차_유효만기_보험료부채_재보험자별 \
    .merge(자동차_유효만기_준비금부채_재보험자별, on=['RINSC_CD', 'KICS_CRD_GRD'], how='outer')
자동차_유효만기_재보험자별[['LIAB_PRM', 'EFF_MAT_PRM', 'LIAB_RSV', 'EFF_MAT_RSV', 'PV_LIAB_PRM_LOSS_ADJ', 'PV_LIAB_RSV_LOSS_ADJ']] = 자동차_유효만기_재보험자별[['LIAB_PRM', 'EFF_MAT_PRM', 'LIAB_RSV', 'EFF_MAT_RSV', 'PV_LIAB_PRM_LOSS_ADJ', 'PV_LIAB_RSV_LOSS_ADJ']].fillna(0)
자동차_유효만기_재보험자별.eval('EFF_MAT = (EFF_MAT_PRM*LIAB_PRM+EFF_MAT_RSV*LIAB_RSV)/(LIAB_PRM+LIAB_RSV)', inplace=True)
자동차_유효만기_재보험자별.eval('PV_LIAB_LOSS_ADJ = PV_LIAB_PRM_LOSS_ADJ+PV_LIAB_RSV_LOSS_ADJ', inplace=True)
자동차_유효만기_재보험자별.eval('LIAB = LIAB_PRM+LIAB_RSV', inplace=True)
자동차_유효만기_재보험자별 = 자동차_유효만기_재보험자별[['RINSC_CD', 'KICS_CRD_GRD', 'EFF_MAT', 'LIAB', 'PV_LIAB_LOSS_ADJ']]

## K-ICS등급별 직전1년출재경과보험료
자동차_출재_직전1년경과보험료_KICS등급별 = pd.concat([자동차_비례출재_직전1년경과보험료_가공, 자동차_비비례출재_직전1년경과보험료_가공], axis=0) \
    .groupby(['KICS_CRD_GRD', 'P_NP_DVCD'], as_index=False)['ELP_PRM'].sum()

In [52]:
# 금리위험액
자동차_현행추정부채_시나리오별 = 자동차_보험료부채_집계.merge(자동차_준비금부채_집계, on=['KICS_SCEN_NO', 'RRNR_DVCD', 'BOZ_CD'], how='outer') \
    .eval('PV_LIAB_PRM_LOSS_ADJ = PV_LIAB_PRM-LOSS_ADJ_PRM') \
    .eval('PV_LIAB_RSV_LOSS_ADJ = PV_LIAB_RSV-LOSS_ADJ_RSV') \
    .eval('PV_LIAB_LOSS_ADJ = PV_LIAB_PRM_LOSS_ADJ+PV_LIAB_RSV_LOSS_ADJ') \
    [['KICS_SCEN_NO', 'RRNR_DVCD', 'BOZ_CD', 'PV_LIAB_PRM_LOSS_ADJ', 'PV_LIAB_RSV_LOSS_ADJ', 'PV_LIAB_LOSS_ADJ']]

In [56]:
# 보험가격준비금위험액
KEYS = ['RRNR_DVCD', 'BOZ_CD', 'CRD_GRD', 'CMSN_DVCD', 'P_NP_DVCD']

## 보험가격익스포져
자동차_보험가격익스포져 = 자동차_직전1년경과보험료_가공.groupby(['RRNR_DVCD', 'BOZ_CD', 'CMSN_DVCD', 'P_NP_DVCD'], as_index=False) \
    [['ELP_PRM']].sum()

## 준비금익스포져
자동차_개별추산액_집계 = 자동차_개별추산액_가공.groupby(KEYS, as_index=False)['OST_AMT'].sum()
자동차_미경과보험료_부분합 = 자동차_미경과보험료_가공 \
    .groupby(['RRNR_DVCD', 'BSC_CVR_CD'])['URND_PRM'].sum().reset_index(name='URND_PRM_SUM')
자동차_미경과보험료_비중 = 자동차_미경과보험료_가공 \
    .merge(자동차_미경과보험료_부분합, on=['RRNR_DVCD', 'BSC_CVR_CD'], how='left') \
    .eval('URND_PRM_WGT = URND_PRM/URND_PRM_SUM')
자동차_IBNR_집계 = 자동차_미경과보험료_비중.merge(자동차_IBNR_가공, on=['RRNR_DVCD', 'BSC_CVR_CD'], how='left') \
    .eval('IBNR = IBNR_TOT*URND_PRM_WGT') \
    .groupby(KEYS, as_index=False)['IBNR'].sum()
자동차_준비금익스포져 = 자동차_개별추산액_집계.merge(자동차_IBNR_집계, on=KEYS, how='outer') \
    .merge(자동차_손해조사비율, on=['RRNR_DVCD', 'BOZ_CD'], how='left') \
    .merge(자동차_할인요소.query('KICS_SCEN_NO == 1')[['BOZ_CD', 'DISC_FAC_RSV']], on=['BOZ_CD'], how='left') \
    .merge(자동차_손실조정율.query('KICS_SCEN_NO == 1')[['RRNR_DVCD', 'BOZ_CD', 'CRD_GRD', 'LOSS_ADJ_RATE_RSV']], on=['RRNR_DVCD', 'BOZ_CD', 'CRD_GRD'], how='left') \
    .assign(OST_AMT = lambda x: x['OST_AMT'].fillna(0).astype(float)) \
    .assign(IBNR = lambda x: x['IBNR'].fillna(0).astype(float)) \
    .assign(LOSS_ADJ_RATE_RSV = lambda x: x['LOSS_ADJ_RATE_RSV'].fillna(0).astype(float))
자동차_준비금익스포져 = 자동차_준비금익스포져 \
    .eval('FUT_LAE = (OST_AMT*0.5+IBNR*1.0)*LAE_RATIO') \
    .eval('LIAB_RSV = OST_AMT+IBNR+FUT_LAE') \
    .eval('PV_OST_AMT = OST_AMT*DISC_FAC_RSV') \
    .eval('PV_IBNR = IBNR*DISC_FAC_RSV') \
    .eval('PV_FUT_LAE = FUT_LAE*DISC_FAC_RSV') \
    .eval('PV_LIAB_RSV = LIAB_RSV*DISC_FAC_RSV') \
    .eval('LOSS_ADJ = LIAB_RSV*LOSS_ADJ_RATE_RSV') \
    .eval('PV_LIAB_RSV_LOSS_ADJ = PV_LIAB_RSV-LOSS_ADJ')
자동차_준비금익스포져_집계 = 자동차_준비금익스포져.groupby(['RRNR_DVCD', 'BOZ_CD', 'CMSN_DVCD', 'P_NP_DVCD'], as_index=False) \
    [['PV_LIAB_RSV_LOSS_ADJ']].sum()

In [58]:
# 데이터 내보내기
now = datetime.now().strftime('%Y%m%d%H%M%S')
with pd.ExcelWriter(f'result/자동차_현행추정부채_위험액_{now}.xlsx') as writer:
    자동차_보험료부채_집계.to_excel(writer, '자동차_보험료부채', index=False)
    자동차_준비금부채_집계.to_excel(writer, '자동차_준비금부채', index=False)
    자동차_보험가격익스포져.to_excel(writer, '자동차_보험가격익스포져', index=False)
    자동차_준비금익스포져_집계.to_excel(writer, '자동차_준비금익스포져', index=False)
    자동차_유효만기_재보험자별.to_excel(writer, '자동차_유효만기_재보험자별', index=False)
    자동차_현행추정부채_시나리오별.to_excel(writer, '자동차_현행추정부채_시나리오별', index=False)
    자동차_출재_직전1년경과보험료_KICS등급별.to_excel(writer, '자동차_출재_직전1년경과보험료_KICS등급별', index=False)